# Lab 3.3.5 Solutions: EAGLE-3 Implementation

Complete solutions to all exercises from the EAGLE-3 implementation lab.

## Setup

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path("../scripts").resolve()))

import numpy as np
import time
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional
import requests

---

## Exercise 1: Implement Feature-Level Speculation Simulator

**Task**: Create a simulator to understand EAGLE's feature-level (not token-level) speculation.

In [ ]:
@dataclass
class EAGLEConfig:
    """Configuration for EAGLE speculation."""
    hidden_size: int = 4096
    draft_hidden_size: int = 1024  # Smaller draft features
    num_draft_layers: int = 2       # Lightweight draft network
    speculation_length: int = 6     # Tokens to speculate
    top_k: int = 10                 # Candidates per position


class EAGLESimulator:
    """
    Simulate EAGLE's feature-level speculative decoding.
    
    KEY DIFFERENCE FROM MEDUSA:
    - Medusa: Each head independently predicts next token
    - EAGLE: Draft model predicts FEATURES, then projects to tokens
    
    This allows EAGLE to capture inter-token dependencies!
    """
    
    def __init__(self, config: EAGLEConfig = None):
        self.config = config or EAGLEConfig()
        
    def simulate_feature_draft(self, 
                                context_features: np.ndarray) -> np.ndarray:
        """
        Simulate the draft network generating feature predictions.
        
        In real EAGLE:
        - Input: Last hidden state from target model
        - Output: Predicted hidden states for next tokens
        
        This is more powerful than token prediction because:
        1. Features encode semantic meaning
        2. Draft model can learn from target model's representations
        3. Better captures long-range dependencies
        """
        batch_size = context_features.shape[0] if len(context_features.shape) > 1 else 1
        
        # Simulate feature predictions (would be neural network in reality)
        # Each position predicts features for the next token
        draft_features = np.zeros((
            batch_size,
            self.config.speculation_length,
            self.config.hidden_size
        ))
        
        for i in range(self.config.speculation_length):
            # Features become less confident further from context
            noise_scale = 0.1 * (1 + i * 0.2)
            if batch_size > 1:
                base = context_features[:, -1, :] if len(context_features.shape) == 3 else context_features
            else:
                base = context_features.flatten()[:self.config.hidden_size]
            
            draft_features[0, i, :len(base)] = base + np.random.randn(len(base)) * noise_scale
        
        return draft_features
    
    def project_features_to_tokens(self,
                                    features: np.ndarray,
                                    vocab_size: int = 32000) -> np.ndarray:
        """
        Project feature predictions to token probabilities.
        
        This is where EAGLE converts feature-level predictions
        to actual token candidates.
        """
        # Simulate projection (would be learned linear layer)
        # Returns: (batch, speculation_length, vocab_size) probabilities
        batch_size, seq_len, hidden = features.shape
        
        # Generate random logits, then softmax
        # In reality, this is a learned projection
        logits = np.random.randn(batch_size, seq_len, vocab_size)
        
        # Apply softmax
        exp_logits = np.exp(logits - logits.max(axis=-1, keepdims=True))
        probs = exp_logits / exp_logits.sum(axis=-1, keepdims=True)
        
        return probs
    
    def get_candidates(self, 
                       token_probs: np.ndarray,
                       top_k: int = None) -> List[List[Tuple[int, float]]]:
        """
        Get top-k token candidates for each position.
        """
        top_k = top_k or self.config.top_k
        batch_size, seq_len, vocab_size = token_probs.shape
        
        candidates = []
        for pos in range(seq_len):
            probs = token_probs[0, pos]
            top_indices = np.argsort(probs)[-top_k:][::-1]
            top_probs = probs[top_indices]
            candidates.append(list(zip(top_indices.tolist(), top_probs.tolist())))
        
        return candidates


# Demonstrate EAGLE mechanics
print("🦅 EAGLE Feature-Level Speculation Simulator")
print("=" * 60)

config = EAGLEConfig(
    hidden_size=256,  # Smaller for demo
    speculation_length=5
)
simulator = EAGLESimulator(config)

# Simulate context features (from target model)
context = np.random.randn(1, 10, config.hidden_size)
print(f"\nContext shape: {context.shape}")
print(f"  (batch=1, seq_len=10, hidden={config.hidden_size})")

# Step 1: Draft features
draft_features = simulator.simulate_feature_draft(context)
print(f"\n📝 Draft Features: {draft_features.shape}")
print(f"  (predicts {config.speculation_length} future positions)")

# Step 2: Project to tokens
token_probs = simulator.project_features_to_tokens(draft_features, vocab_size=1000)
print(f"\n🎯 Token Probabilities: {token_probs.shape}")

# Step 3: Get candidates
candidates = simulator.get_candidates(token_probs, top_k=3)
print(f"\n📋 Top-3 Candidates per position:")
for pos, cands in enumerate(candidates):
    cand_str = [(f"token_{c[0]}", f"{c[1]:.3f}") for c in cands]
    print(f"  Position {pos+1}: {cand_str}")

print("\n💡 Key Insight: EAGLE predicts FEATURES first, then tokens.")
print("   This captures inter-token dependencies better than Medusa!")

---

## Exercise 2: Compare EAGLE vs Medusa

**Task**: Analyze the theoretical and practical differences between EAGLE and Medusa.

In [ ]:
@dataclass
class MethodComparison:
    """Comparison between speculation methods."""
    method: str
    param_count_mb: float
    forward_cost_ratio: float  # Relative to base model
    avg_acceptance: float
    tokens_per_step: float
    effective_speedup: float


def analyze_method(method: str,
                   base_model_params_b: float = 8,  # 8B parameters
                   num_heads: int = 5,
                   head_size_ratio: float = 0.01,  # Medusa head size
                   draft_size_ratio: float = 0.05) -> MethodComparison:  # EAGLE draft size
    """
    Analyze a speculative decoding method.
    """
    if method == "medusa":
        # Medusa: Multiple small heads
        param_count = base_model_params_b * head_size_ratio * num_heads * 1000  # MB
        forward_cost = 0.05 * num_heads  # ~5% per head
        
        # Acceptance decreases per head (independent predictions)
        base_acceptance = 0.75
        expected_tokens = sum(
            base_acceptance * (0.85 ** i) for i in range(num_heads)
        )
        
    elif method == "eagle":
        # EAGLE: One draft model
        param_count = base_model_params_b * draft_size_ratio * 1000  # MB
        forward_cost = 0.08  # ~8% for draft model
        
        # EAGLE has better acceptance due to feature-level prediction
        base_acceptance = 0.82  # Higher than Medusa
        expected_tokens = sum(
            base_acceptance * (0.90 ** i) for i in range(num_heads)  # Slower decay
        )
        
    elif method == "eagle3":
        # EAGLE-3: Improved architecture
        param_count = base_model_params_b * draft_size_ratio * 0.8 * 1000  # Smaller
        forward_cost = 0.06  # More efficient
        
        # EAGLE-3 has even better acceptance
        base_acceptance = 0.88
        expected_tokens = sum(
            base_acceptance * (0.92 ** i) for i in range(num_heads)
        )
    else:
        raise ValueError(f"Unknown method: {method}")
    
    tokens_per_step = 1 + expected_tokens
    effective_speedup = tokens_per_step / (1 + forward_cost)
    
    return MethodComparison(
        method=method,
        param_count_mb=param_count,
        forward_cost_ratio=forward_cost,
        avg_acceptance=base_acceptance,
        tokens_per_step=tokens_per_step,
        effective_speedup=effective_speedup
    )


# Compare methods
print("📊 EAGLE vs Medusa Comparison")
print("=" * 75)
print(f"{'Method':<12} {'Params':>10} {'Overhead':>10} {'Acceptance':>12} {'Tok/Step':>10} {'Speedup':>10}")
print("-" * 75)

methods = ["medusa", "eagle", "eagle3"]
results = []

for method in methods:
    result = analyze_method(method)
    results.append(result)
    
    print(f"{result.method:<12} {result.param_count_mb:>8.0f}MB "
          f"{result.forward_cost_ratio:>9.0%} {result.avg_acceptance:>11.0%} "
          f"{result.tokens_per_step:>10.2f} {result.effective_speedup:>9.2f}x")

print("\n📋 Key Differences:")
print("""
┌─────────────────┬─────────────────────────┬─────────────────────────┐
│ Aspect          │ Medusa                  │ EAGLE                   │
├─────────────────┼─────────────────────────┼─────────────────────────┤
│ Prediction      │ Token-level (logits)    │ Feature-level (hidden)  │
│ Architecture    │ Multiple small heads    │ Single draft model      │
│ Dependencies    │ Independent per head    │ Captures inter-token    │
│ Training        │ Simple, parallel        │ More complex, KD        │
│ Acceptance      │ Lower, fast decay       │ Higher, slow decay      │
│ Best for        │ Simple patterns         │ Complex dependencies    │
└─────────────────┴─────────────────────────┴─────────────────────────┘
""")

best = max(results, key=lambda x: x.effective_speedup)
print(f"✅ Winner: {best.method.upper()} with {best.effective_speedup:.2f}x speedup")

---

## Exercise 3: Implement Adaptive Tree Construction

**Task**: Build a dynamic speculation tree that adapts based on acceptance patterns.

In [ ]:
@dataclass
class TreeNode:
    """Node in the speculation tree."""
    token_id: int
    probability: float
    depth: int
    children: List['TreeNode'] = field(default_factory=list)
    path: List[int] = field(default_factory=list)


class AdaptiveTreeBuilder:
    """
    Build speculation trees that adapt based on observed patterns.
    
    EAGLE-3 uses learned tree structures rather than fixed ones.
    This simulator shows how adaptive trees improve efficiency.
    """
    
    def __init__(self,
                 max_depth: int = 6,
                 max_candidates: int = 64,
                 prune_threshold: float = 0.05):
        self.max_depth = max_depth
        self.max_candidates = max_candidates
        self.prune_threshold = prune_threshold
        
        # Track acceptance by depth and branching factor
        self.depth_acceptance: Dict[int, List[float]] = {i: [] for i in range(max_depth)}
        self.branching_acceptance: Dict[int, List[float]] = {i: [] for i in range(1, 6)}
    
    def build_tree(self,
                   token_probs: List[List[Tuple[int, float]]],
                   strategy: str = "adaptive") -> Tuple[TreeNode, int]:
        """
        Build a speculation tree from token probabilities.
        
        Strategies:
        - fixed: Fixed branching factor at each level
        - greedy: More branches for high-probability tokens
        - adaptive: Learn branching from acceptance history
        """
        root = TreeNode(token_id=-1, probability=1.0, depth=0)
        total_nodes = 0
        
        def get_branching_factor(depth: int, parent_prob: float) -> int:
            if strategy == "fixed":
                return 3
            elif strategy == "greedy":
                # More branches for high-probability paths
                return min(5, max(1, int(parent_prob * 5)))
            elif strategy == "adaptive":
                # Use historical acceptance to determine branching
                if self.depth_acceptance[depth]:
                    avg_acceptance = np.mean(self.depth_acceptance[depth][-20:])
                    # Higher acceptance = more branches worthwhile
                    return min(5, max(1, int(avg_acceptance * 6)))
                return 3  # Default
            else:
                return 3
        
        def build_level(parent: TreeNode, depth: int, cumulative_prob: float):
            nonlocal total_nodes
            
            if depth >= min(len(token_probs), self.max_depth):
                return
            if total_nodes >= self.max_candidates:
                return
            if cumulative_prob < self.prune_threshold:
                return  # Prune low-probability paths
            
            branching = get_branching_factor(depth, cumulative_prob)
            candidates = token_probs[depth][:branching]
            
            for token_id, prob in candidates:
                if total_nodes >= self.max_candidates:
                    break
                
                child = TreeNode(
                    token_id=token_id,
                    probability=prob,
                    depth=depth + 1,
                    path=parent.path + [token_id]
                )
                parent.children.append(child)
                total_nodes += 1
                
                # Recursively build subtree
                build_level(child, depth + 1, cumulative_prob * prob)
        
        build_level(root, 0, 1.0)
        return root, total_nodes
    
    def record_acceptance(self, accepted_path: List[int], full_path: List[int]):
        """Record acceptance for adaptive learning."""
        for depth in range(len(full_path)):
            accepted = depth < len(accepted_path)
            self.depth_acceptance[depth].append(1.0 if accepted else 0.0)
    
    def get_tree_stats(self, root: TreeNode) -> dict:
        """Get statistics about a tree."""
        nodes_by_depth = {}
        all_paths = []
        
        def traverse(node: TreeNode):
            depth = node.depth
            nodes_by_depth[depth] = nodes_by_depth.get(depth, 0) + 1
            
            if not node.children:
                all_paths.append(node.path)
            for child in node.children:
                traverse(child)
        
        traverse(root)
        
        return {
            "nodes_by_depth": nodes_by_depth,
            "total_paths": len(all_paths),
            "avg_depth": np.mean([len(p) for p in all_paths]) if all_paths else 0,
            "max_depth": max(len(p) for p in all_paths) if all_paths else 0
        }


# Demonstrate adaptive tree construction
print("🌲 Adaptive Speculation Tree Builder")
print("=" * 60)

builder = AdaptiveTreeBuilder(max_depth=5, max_candidates=32)

# Simulate token probabilities
np.random.seed(42)
token_probs = []
for depth in range(5):
    # Probabilities decay with depth
    probs = np.random.dirichlet(np.ones(5) * (2 - depth * 0.3))
    probs = sorted(probs, reverse=True)
    token_probs.append([(i, p) for i, p in enumerate(probs)])

# Build trees with different strategies
strategies = ["fixed", "greedy", "adaptive"]

print(f"\n{'Strategy':<12} {'Nodes':>8} {'Paths':>8} {'Avg Depth':>10} {'Max Depth':>10}")
print("-" * 55)

for strategy in strategies:
    root, total_nodes = builder.build_tree(token_probs, strategy=strategy)
    stats = builder.get_tree_stats(root)
    
    print(f"{strategy:<12} {total_nodes:>8} {stats['total_paths']:>8} "
          f"{stats['avg_depth']:>10.1f} {stats['max_depth']:>10}")

# Simulate adaptive learning
print("\n🎓 Simulating Adaptive Learning...")

# Simulate some generations with varying acceptance
for _ in range(50):
    # Simulate acceptance (higher at lower depths)
    accepted_len = np.random.choice([1, 2, 3, 4, 5], p=[0.1, 0.2, 0.3, 0.25, 0.15])
    builder.record_acceptance(
        accepted_path=list(range(accepted_len)),
        full_path=list(range(5))
    )

# Build with learned patterns
root_adaptive, nodes_adaptive = builder.build_tree(token_probs, strategy="adaptive")
stats_adaptive = builder.get_tree_stats(root_adaptive)

print(f"\n📊 After learning (50 samples):")
print(f"   Nodes: {nodes_adaptive}")
print(f"   Avg depth: {stats_adaptive['avg_depth']:.1f}")
print(f"   Acceptance by depth: ", end="")
for d in range(5):
    if builder.depth_acceptance[d]:
        print(f"D{d}={np.mean(builder.depth_acceptance[d]):.0%} ", end="")
print()

print("\n💡 Adaptive tree adjusts branching based on observed acceptance!")

---

## Exercise 4: Benchmark EAGLE on Different Workloads

**Task**: Analyze EAGLE performance across different types of generation tasks.

In [ ]:
@dataclass
class WorkloadResult:
    """Results from benchmarking a workload."""
    workload: str
    base_tokens_per_sec: float
    eagle_tokens_per_sec: float
    speedup: float
    avg_accepted: float
    tree_efficiency: float


# Workload characteristics
WORKLOADS = {
    "code_generation": {
        "description": "Generate Python/JS code",
        "predictability": 0.9,  # High - syntax is predictable
        "entropy": 0.3,
        "typical_length": 200,
    },
    "json_generation": {
        "description": "Generate structured JSON",
        "predictability": 0.95,  # Very high - fixed structure
        "entropy": 0.2,
        "typical_length": 150,
    },
    "chat_response": {
        "description": "Conversational responses",
        "predictability": 0.7,
        "entropy": 0.5,
        "typical_length": 100,
    },
    "creative_story": {
        "description": "Creative fiction writing",
        "predictability": 0.5,
        "entropy": 0.8,
        "typical_length": 300,
    },
    "math_reasoning": {
        "description": "Step-by-step math solutions",
        "predictability": 0.75,
        "entropy": 0.4,
        "typical_length": 250,
    },
    "translation": {
        "description": "Language translation",
        "predictability": 0.85,
        "entropy": 0.35,
        "typical_length": 120,
    }
}


def simulate_eagle_benchmark(workload: str,
                              base_tps: float = 50,
                              speculation_depth: int = 5) -> WorkloadResult:
    """
    Simulate EAGLE performance on a specific workload.
    """
    config = WORKLOADS[workload]
    
    # Calculate expected acceptance based on workload characteristics
    # EAGLE does better with predictable, low-entropy text
    base_acceptance = config["predictability"] * 0.95
    
    # Calculate tokens per step
    expected_tokens = 1  # Base token
    cumulative_prob = 1.0
    
    for i in range(speculation_depth):
        # Acceptance decays with depth and entropy
        step_acceptance = base_acceptance * (0.92 ** i) * (1 - config["entropy"] * 0.2)
        cumulative_prob *= step_acceptance
        expected_tokens += cumulative_prob
    
    # Tree efficiency: how many candidates are actually used
    # Higher predictability = fewer wasted candidates
    tree_efficiency = config["predictability"] * 0.9
    
    # Calculate effective speedup (accounting for draft overhead)
    draft_overhead = 0.08  # ~8% for EAGLE draft model
    effective_speedup = expected_tokens / (1 + draft_overhead)
    
    eagle_tps = base_tps * effective_speedup
    
    return WorkloadResult(
        workload=workload,
        base_tokens_per_sec=base_tps,
        eagle_tokens_per_sec=eagle_tps,
        speedup=effective_speedup,
        avg_accepted=expected_tokens - 1,  # Exclude base token
        tree_efficiency=tree_efficiency
    )


# Run benchmarks
print("📊 EAGLE Workload Benchmark")
print("=" * 80)
print(f"{'Workload':<18} {'Base TPS':>10} {'EAGLE TPS':>12} {'Speedup':>10} {'Avg Accept':>12}")
print("-" * 80)

results = []
for workload in WORKLOADS:
    result = simulate_eagle_benchmark(workload)
    results.append(result)
    
    bar = "█" * int(result.speedup * 2)
    print(f"{workload:<18} {result.base_tokens_per_sec:>10.0f} {result.eagle_tokens_per_sec:>12.0f} "
          f"{result.speedup:>9.2f}x {result.avg_accepted:>12.1f} {bar}")

# Analysis
best = max(results, key=lambda x: x.speedup)
worst = min(results, key=lambda x: x.speedup)
avg_speedup = np.mean([r.speedup for r in results])

print(f"\n📈 Summary:")
print(f"   Best: {best.workload} ({best.speedup:.2f}x)")
print(f"   Worst: {worst.workload} ({worst.speedup:.2f}x)")
print(f"   Average: {avg_speedup:.2f}x")

print("\n💡 Key Insights:")
print("   • JSON/Code generation: EAGLE excels (predictable patterns)")
print("   • Creative writing: Lower speedup (high entropy)")
print("   • Consider workload mix when evaluating overall benefit")
print("   • EAGLE's feature-level prediction helps with structured outputs")

---

## Key Takeaways

1. **Feature-Level Prediction**: EAGLE predicts hidden states, not just tokens
   - Captures inter-token dependencies
   - Better acceptance rates than Medusa

2. **EAGLE vs Medusa**:
   - EAGLE: Single draft model, higher accuracy, slightly more overhead
   - Medusa: Multiple heads, simpler training, faster per-head
   - EAGLE typically wins for complex, dependent outputs

3. **Adaptive Trees**:
   - Learn branching factors from acceptance patterns
   - Prune low-probability paths
   - Dynamically adjust to content characteristics

4. **Workload Matters**:
   - Structured outputs (JSON, code): High speedup
   - Creative writing: Lower speedup
   - Match speculation strategy to your workload